# Analysis
## Target
In the last notebook 99.4%+ was achieved 4 times between 10th and 15th epoch. Now the aim of this iteration is to reduce the parameters to levels below 10000. We will do the same by reducing the kernel size across the layers with minimal impact on accuracy. For the purpose of the same, I'm planning to include additional convolutional layers in the various blocks

## Result
Best Training Accuracy achieved in Epoch 14: 97.97%     
Best Test Accuracy is achieved in Epoch 14: 99.12%

## Analysis
The desired goal of reducing the parameters is achieved by reducing the kernel sizes and a slight increase in layers. This has reduced the accuracy as expected. Now the focus can be in increasing the accuracy by introducing the image augmentation.


In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(4)
        self.dropout1 = nn.Dropout(0.15)
        self.conv2 = nn.Conv2d(4, 4, 3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(4)
        self.dropout2 = nn.Dropout(0.15)
        self.conv3 = nn.Conv2d(4, 4, 3, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(4)
        self.dropout3 = nn.Dropout(0.15)
        self.conv3a = nn.Conv2d(4, 4, 3, padding=1)
        self.batchnorm3a = nn.BatchNorm2d(4)
        self.dropout3a = nn.Dropout(0.15)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(4, 8, 3, padding=1)
        self.batchnorm4 = nn.BatchNorm2d(8)
        self.dropout4 = nn.Dropout(0.15)
        self.conv5 = nn.Conv2d(8, 8, 3, padding=1)
        self.batchnorm5 = nn.BatchNorm2d(8)
        self.dropout5 = nn.Dropout(0.15)
        self.conv6 = nn.Conv2d(8, 8, 3, padding=1)
        self.batchnorm6 = nn.BatchNorm2d(8)
        self.dropout6 = nn.Dropout(0.15)
        self.conv6a = nn.Conv2d(8, 8, 3, padding=1)
        self.batchnorm6a = nn.BatchNorm2d(8)
        self.dropout6a = nn.Dropout(0.15)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv7 = nn.Conv2d(8, 16, 3)
        self.batchnorm7 = nn.BatchNorm2d(16)
        self.dropout7 = nn.Dropout(0.15)
        self.conv8 = nn.Conv2d(16, 16, 3)
        self.batchnorm8= nn.BatchNorm2d(16)
        self.dropout8 = nn.Dropout(0.15)
        self.conv8a = nn.Conv2d(16, 16, 3)
        self.batchnorm8a= nn.BatchNorm2d(16)
        self.dropout8a = nn.Dropout(0.15)
        self.conv9 = nn.Conv2d(16, 10, 1)

    def forward(self, x):
        x = self.pool1(self.dropout3a(self.batchnorm3a(F.relu(self.conv3a(self.dropout3(self.batchnorm3(F.relu(self.conv3(self.dropout2(self.batchnorm2(F.relu(self.conv2(self.dropout1(self.batchnorm1(F.relu(self.conv1(x)))))))))))))))))
        x = self.pool2(self.dropout6a(self.batchnorm6a(F.relu(self.conv6a(self.dropout6(self.batchnorm6(F.relu(self.conv6(self.dropout5(self.batchnorm5(F.relu(self.conv5(self.dropout4(self.batchnorm4(F.relu(self.conv4(x)))))))))))))))))
        x = self.dropout8a(self.batchnorm8a(F.relu(self.conv8a(self.dropout8(self.batchnorm8(F.relu(self.conv8(self.dropout7(self.batchnorm7(F.relu(self.conv7(x))))))))))))
#        x = F.relu(self.conv9(x))
        x = self.conv9(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [21]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
       BatchNorm2d-2            [-1, 4, 28, 28]               8
           Dropout-3            [-1, 4, 28, 28]               0
            Conv2d-4            [-1, 4, 28, 28]             148
       BatchNorm2d-5            [-1, 4, 28, 28]               8
           Dropout-6            [-1, 4, 28, 28]               0
            Conv2d-7            [-1, 4, 28, 28]             148
       BatchNorm2d-8            [-1, 4, 28, 28]               8
           Dropout-9            [-1, 4, 28, 28]               0
           Conv2d-10            [-1, 4, 28, 28]             148
      BatchNorm2d-11            [-1, 4, 28, 28]               8
          Dropout-12            [-1, 4, 28, 28]               0
        MaxPool2d-13            [-1, 4, 14, 14]               0
           Conv2d-14            [-1, 8,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    print('\nTrain set: Accuracy: {}/{} ({:.2f}%)\n'.format(
        correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [24]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer,step_size=2,gamma=0.45)

for epoch in range(1, 16):
    print("\n EPOC NUMBER IS :", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    print("\n--------------------------------------------------------------------------- \n")

  0%|          | 0/938 [00:00<?, ?it/s]


 EPOC NUMBER IS : 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.34865909814834595 batch_id=937: 100%|██████████| 938/938 [00:18<00:00, 51.65it/s]


Train set: Accuracy: 53043/60000 (88.41%)




  0%|          | 0/938 [00:00<?, ?it/s]

...


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0282, Accuracy: 9903/10000 (99.03%)


--------------------------------------------------------------------------- 


 EPOC NUMBER IS : 15


loss=0.1590227484703064 batch_id=937: 100%|██████████| 938/938 [00:18<00:00, 49.90it/s]


Train set: Accuracy: 58779/60000 (97.97%)




Test set: Average loss: 0.0266, Accuracy: 9912/10000 (99.12%)


--------------------------------------------------------------------------- 

